In [1]:
from __future__ import division

In [2]:
import math
import numpy as np
import pandas as pd

## Concepts:

### Tree
A Tree is an object that takes input data and determines what leaf it ends up in.  Unlike many tree implementations, the Tree itself doesn't store data about the value of a leaf.  That is stored externally.

### leaf_score_map
A leaf_score_map is a map of leaf ids (eg their hash) to the score/value for that leaf.  One can only use a tree to score data if one has a leaf_score_map.  This design allows on to use the same tree as a subset of another tree without having their leaf values become entangled

### loss_fn
A loss_fn is a function that takes data rows, the predicted targets for those rows, and the actual targets for those rows, and returns a single value that determines the "LOSS" or "COST" of that prediction (lower cost/loss is better)

```
def loss_fn(features, actual_targets, predicted_targets)
```



### leaf_value_fn
A leaf_value_fn is a function of the features and actual targets that end up in a leaf to the prediction of that leaf.  It is typically either the mean good rate in that leaf (among the actual targets) or the median target, but can be anything else

```
def leaf_value_fn(features, actual_targets)
```

In [3]:
from abc import ABCMeta, abstractmethod


class Node(object):
    __metaclass__ = ABCMeta
    
    @abstractmethod
    def find_leaves(self, df):
        raise NotImplemented()
        
    @abstractmethod
    def prn(self, indent=None):
        raise NotImplemented()
            
    def predict(self, df, leaf_score_map):
        return self.find_leaves(df).map(lambda l: leaf_score_map[l])
        

class BranchNode(Node):
    
    def __init__(self, var_name, split, left, right):
        self.var_name = var_name
        self.split = split
        self.left = left
        self.right = right
        
    def find_leaves(self, df):
        idx_left = (df[self.var_name] <= self.split)        
        left_leaves = self.left.find_leaves(df[idx_left])            
        right_leaves = self.right.find_leaves(df[~idx_left])                
        return pd.concat([left_leaves, right_leaves]).loc[df.index]
    
    def prn(self, indent=None):
        
        indent = indent or 0
        
        if self.left:
            self.left.prn(indent+1)
        
        for _ in range(indent):
            print '\t',
        print "{} {}\n".format(self.var_name, self.split)
            
        if self.right:
            self.right.prn(indent+1)
                    
    
class LeafNode(Node):
    
    def __init__(self):
        pass
        
    def find_leaves(self, df):
        return pd.Series(hash(self), index=df.index)
    
    def prn(self, indent=None):
        
        indent = indent or 0
        
        for _ in range(indent):
            print '\t',
        print "Leaf({})\n".format(hash(self))

In [4]:
def _single_variable_best_split(df, var, target, loss_fn):
    
    # Convention:
    # Left is BAD
    # Right is GOOD
    
    srs = df[var]
    
    if len(srs) < 100:
        candidates = srs.values
    else:
        _, candidates = pd.qcut(srs, 100, labels=False, retbins=True)
    
    best_loss = None
    best_split = None
    
    for val in candidates:
        left_idx = (srs <= val)
                
        left_truth = target[left_idx]
        left_predicted = [left_truth.mean() for _ in left_truth]
                
        right_truth = target[~left_idx]
        right_predicted = [right_truth.mean() for _ in right_truth]
                
        loss = loss_fn(df.loc[left_truth.index], left_truth, left_predicted) + loss_fn(df.loc[right_truth.index], right_truth, right_predicted)
        
        if best_loss is None or loss < best_loss:
            best_split = val
            best_loss = loss
            
    return best_split, best_loss
 

def get_best_split(df, target, loss_fn):
    # Return:
    # (var, split, loss)
    
    best_var = None
    best_split = None
    best_loss = None
    
    for var in df.columns:
        split, loss = _single_variable_best_split(df, var, target, loss_fn)
        if best_loss is None or loss < best_loss:
            best_var = var
            best_split = split
            best_loss = loss
            
    return (best_var, best_split, best_loss)    

In [5]:
def leaf_good_rate(features, target):
    counts = dict(target.value_counts())
    num_good = counts.get(1, 0)
    num_bad = counts.get(0, 0)
    return num_good / (num_good + num_bad)
#    return num_good / (num_bad + num_good)
    

def train_greedy_tree(df, target, loss_fn,
                      max_depth=None,
                      min_to_split=None,
                      leaf_map=None,
                      leaf_value_fn=leaf_good_rate):
    """
    Returns a tree and its leaf map
    """
    
    if leaf_map is None:
        leaf_map = {}
    
    current_loss = loss_fn(df, target, [leaf_good_rate(df, target) for _ in target])
    
    if len(df) == 1 or (max_depth is not None and max_depth <= 0) or (min_to_split is not None and len(df) < min_to_split):
        leaf = LeafNode()
        leaf_map[hash(leaf)] = leaf_value_fn(df, target)
        return leaf, leaf_map
    
    var, split, loss = get_best_split(df, target, loss_fn)

    if loss >= current_loss:
        leaf = LeafNode()
        leaf_map[hash(leaf)] = leaf_value_fn(df, target)
        return leaf, leaf_map
    
    left_idx = df[var] <= split
    
    left_tree, left_map = train_greedy_tree(df[left_idx], target[left_idx],
                                            loss_fn,
                                            max_depth = max_depth-1 if max_depth else None,
                                            min_to_split=min_to_split,
                                            leaf_map=leaf_map,
                                            leaf_value_fn=leaf_value_fn)
                                  
    right_tree, right_map = train_greedy_tree(df[~left_idx], target[~left_idx],
                                              loss_fn,
                                              max_depth = max_depth-1 if max_depth else None,
                                              min_to_split=min_to_split,
                                              leaf_map=leaf_map,
                                              leaf_value_fn=leaf_value_fn)
    
    leaf_map.update(left_map)
    leaf_map.update(right_map)
    
    return (BranchNode(var, split,
                      left_tree, right_tree),
            leaf_map)

In [6]:
def calculate_leaf_map(tree, df, target, leaf_value_fn=leaf_good_rate):
    
    leaf_map = {}
    
    leaves = tree.find_leaves(df)
    
    for leaf_hash, leaf_rows in df.groupby(leaves):
        
        
        leaf_map[leaf_hash] = leaf_value_fn(leaf_rows,
                                            target.loc[leaf_rows.index])
    
    return leaf_map

In [7]:
def mate(mother, father):
    """
    Create a child tree
    """
    
    
    

In [8]:
data = pd.DataFrame({'A': [0.1, 10, .02],
                     'B': [10, 20, 30]},
                    index=['foo', 'bar', 'baz'])

In [9]:
t = BranchNode('A', 0.5, None, None)
t.left = LeafNode() #'A', 0.5, 10, 20)
t.right = LeafNode() #'A', 0.5, 100, 0)


leaf_map = {hash(t.left): 10,
            hash(t.right): 20}

t.predict(data, leaf_map)

foo    10
bar    20
baz    10
dtype: int64

In [10]:
def cut(x, min, max):
    if x < min:
        return min
    elif x > max:
        return max
    else:
        return x

def cross_entropy(df, truth, predicted, threshold=0.5):
    
    if len(predicted) == 0:
        return 0
    
    loss = 0
    
    for (p, t) in zip(predicted, truth):
    
        # Correct prediction
        if (p >= threshold and t==1) or (p < threshold and t==0):
            loss += math.log(cut(p, 0.001, 0.999))
        else:
            loss += math.log(cut(1.0-p, 0.001, 0.999))
        
    return loss

In [11]:
df = pd.DataFrame({'foo': pd.Series([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])})

_single_variable_best_split(df,
                            'foo',
                            pd.Series([0, 0, 1, 0, 0, 1, 1, 0, 1, 1]),
                            cross_entropy)

(2, -19.108016463228132)

In [12]:
df = pd.DataFrame({'A': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
                   'B': [10, 20, 50, 30, 40, 50, 60, 50, 70, 90, 100, 110 ]})
target = pd.Series([0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0])

tree, leaf_map = train_greedy_tree(df, target, cross_entropy)

In [13]:
tree.prn()

	Leaf(285868745)

B 40

		Leaf(285861181)

	A 10

		Leaf(285865893)



In [14]:
leaf_map

{285861181: 0.83333333333333337, 285865893: 0.0, 285868745: 0.0}

In [15]:
calculate_leaf_map(tree, df, target)

{285861181: 0.83333333333333337, 285865893: 0.0, 285868745: 0.0}